<a href="https://colab.research.google.com/github/BimaAfka14/Sentiment-Analisis-Machine-Learning/blob/master/Crawl_Komentar_Youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pull Comment

In [ ]:
!pip install google-api-python-client pandas

Output as Pandas DataFrame

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re
import googleapiclient.discovery
import pandas as pd
from google.colab import drive

# Function to extract video ID from YouTube URL
def extract_video_id(url):
    # Regular expression pattern to match YouTube video ID
    pattern = r"(?<=v=)[a-zA-Z0-9_-]+(?=&|\?|$)"
    match = re.search(pattern, url)
    if match:
        return match.group(0)
    else:
        return None

api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY =  # Ganti dengan kunci API Anda
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY)

# Hubungkan ke Google Drive
drive.mount('/content/drive')

# Meminta pengguna memasukkan tautan video YouTube
video_url = input("Masukkan tautan video YouTube: ")
video_id = extract_video_id(video_url)

if video_id:
    comments = []
    page_token = None

    while True:
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=100,
            pageToken=page_token
        )
        response = request.execute()

        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments.append({
                'Timestamp': comment['publishedAt'],
                'Username': comment['authorDisplayName'],
                'VideoID': video_id,
                'Comment': comment['textDisplay'],
                'Date': comment['updatedAt'] if 'updatedAt' in comment else comment['publishedAt']
            })

        page_token = response.get('nextPageToken')

        if not page_token:
            break

    df = pd.DataFrame(comments)
    # Simpan file CSV ke Google Drive
    df.to_csv('/content/drive/MyDrive/Colab Notebooks/data_debat/youtube_video_comments.csv', index=False)

    print("Total comments fetched:", len(comments))
    df.head()
else:
    print("Tautan video YouTube tidak valid. Pastikan Anda memasukkan tautan yang benar.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Masukkan tautan video YouTube: https://www.youtube.com/watch?v=Ua21uPr1eRc&t=2s
Total comments fetched: 4494


In [ ]:
# from google.colab import files
# files.download('youtube_comments.csv')